In [1]:
import pandas as pd
import numpy as np

## All Data

In [2]:
df = pd.read_csv('data/nfl_combine_2010_to_2023.csv')
df.head()

,Year,Player,Pos,School,Height,Weight,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted,Round,Pick
0,2010,Seyi Ajirotutu,WR,Fresno State,6-3,204.0,4.60,36.0,14.0,115.0,7.22,4.39,False,NaN,NaN
1,2010,Rahim Alem,DE,LSU,6-3,251.0,4.75,30.5,NaN,106.0,7.54,4.80,False,NaN,NaN
2,2010,Charles Alexander,DT,LSU,6-4,300.0,5.40,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
3,2010,Danario Alexander,WR,Missouri,6-5,215.0,4.62,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
4,2010,Nate Allen,S,South Florida,6-0,207.0,4.50,NaN,16.0,NaN,NaN,NaN,True,2.0,37.0


### Lets Create a new DF that shows if a player participated or not so we could possibly create a model to predict if a player is drafted or not

In [3]:
# First get a list of all the columns
columns = ['Height', 'Weight', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle']
#Then create a DNP column for each column, standing for did not participate
for column in columns:
    dnp_column = f'{column}_dnp'

    #Set the value of the dnp column to 1 if it is null or 0 if it is not
    df[dnp_column] = df[column].isna().astype(int)

    #Replace nulls with -1
    df.fillna({column:-1}, inplace=True)
df.head()

,Year,Player,Pos,School,Height,Weight,40yd,Vertical,Bench,Broad Jump,...,Round,Pick,Height_dnp,Weight_dnp,40yd_dnp,Vertical_dnp,Bench_dnp,Broad Jump_dnp,3Cone_dnp,Shuttle_dnp
0,2010,Seyi Ajirotutu,WR,Fresno State,6-3,204.0,4.60,36.0,14.0,115.0,...,NaN,NaN,0,0,0,0,0,0,0,0
1,2010,Rahim Alem,DE,LSU,6-3,251.0,4.75,30.5,-1.0,106.0,...,NaN,NaN,0,0,0,0,1,0,0,0
2,2010,Charles Alexander,DT,LSU,6-4,300.0,5.40,-1.0,-1.0,-1.0,...,NaN,NaN,0,0,0,1,1,1,1,1
3,2010,Danario Alexander,WR,Missouri,6-5,215.0,4.62,-1.0,-1.0,-1.0,...,NaN,NaN,0,0,0,1,1,1,1,1
4,2010,Nate Allen,S,South Florida,6-0,207.0,4.50,-1.0,16.0,-1.0,...,2.0,37.0,0,0,0,1,0,1,1,1


### Save to a csv to use later

In [4]:
df.to_csv('data/draft_prediction.csv', index=False)

## Lets prep Drafted vs Undrafted players into seperate dataframes

In [6]:
drafted = df[df['Drafted'] == True].copy()
undrafted = df[df['Drafted'] == False].copy()

### Height will be an issue for aggregation because its an object set up like '6-5', '5-11', etc.

Lets build a functions to convert to inches then back

In [7]:
def height_to_inches(height):
    if height == -1:
        return -1
    feet, inches = map(int, height.split('-'))
    return feet * 12 + inches

In [8]:
def inches_to_height(inches):
    if inches == -1:
        return -1
    feet = int(inches//12)
    remaining_inches = int(inches % 12)
    return f'{feet}-{remaining_inches}'

In [9]:
drafted['Height_in_inches'] = drafted['Height'].apply(height_to_inches)
undrafted['Height_in_inches'] = undrafted['Height'].apply(height_to_inches)

### Lets generate the average stats for each position each year so we can see how each position compares in each event later

In [10]:
positions = list(drafted['Pos'].unique())
averages = ['Height_in_inches','Weight', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle']

In [15]:
positions

['S',
 'DE',
 'ILB',
 'CB',
 'OG',
 'DT',
 'WR',
 'RB',
 'P',
 'OLB',
 'QB',
 'OT',
 'TE',
 'FB',
 'C',
 'K',
 'LS',
 'EDGE',
 'LB',
 'DB',
 'OL',
 'DL']

In [21]:
def generate_averages_df(df):
    result_list = []

    for pos in positions:
        # Filter for the current position
        pos_df = df[df['Pos'] == pos]

        for event in averages:
        # Exclude players who did not participate in the event
            event_df = pos_df[pos_df[event] != -1]

        # Get the average of each event per year
            yearly_avg = round(event_df.groupby('Year')[event].mean(),2).to_dict()

        # Append results to the list
            for year, avg in yearly_avg.items():
            # Find if there's an existing entry for the same year and position
                existing_entry = next((item for item in result_list if item['Year'] == year and item['Position'] == pos), None)
                if existing_entry:
                # If found, update the entry with the new event average
                    existing_entry[event] = avg
                else:
                # If not found, create a new entry
                    result_list.append({
                        'Year': year,
                        'Position': pos,
                        event: avg
                    })

    # Convert the result list into a DataFrame
    result_df = pd.DataFrame(result_list)
    result_df['Height'] = result_df['Height_in_inches'].apply(inches_to_height)
    result_df.fillna(0, inplace=True)
    return result_df

In [22]:
undrafted_averages = generate_averages_df(undrafted)
drafted_averages = generate_averages_df(drafted)

In [24]:
undrafted_averages

,Year,Position,Height_in_inches,Weight,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Height
0,2010,S,71.91,208.64,4.59,34.44,15.89,118.30,6.98,4.25,5-11
1,2011,S,71.75,207.62,4.56,33.92,19.20,118.17,6.91,4.19,5-11
2,2012,S,71.90,206.50,4.56,34.40,17.78,121.40,7.00,4.24,5-11
3,2013,S,72.17,210.33,4.62,35.40,16.00,121.80,6.87,4.28,6-0
4,2014,S,72.00,208.20,4.66,34.30,16.00,119.00,6.91,4.23,6-0
...,...,...,...,...,...,...,...,...,...,...,...
227,2021,OL,75.75,299.84,5.29,27.48,23.13,102.82,7.81,4.82,6-3
228,2016,DL,73.33,325.33,5.25,23.50,33.00,104.00,0.00,5.13,6-1
229,2019,DL,74.60,290.60,5.16,27.62,27.75,109.00,7.67,4.61,6-2
230,2020,DL,74.60,283.00,5.01,28.80,22.14,109.44,7.75,4.80,6-2


In [19]:
drafted_averages

,Year,Position,Height_in_inches,Weight,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Height
0,2010,S,72.44,210.50,4.52,36.92,16.94,120.38,6.97,4.29,6-0
1,2011,S,72.15,208.08,4.59,33.19,17.62,117.00,6.89,4.11,6-0
2,2012,S,72.38,208.85,4.54,35.10,17.70,123.00,6.94,4.21,6-0
3,2013,S,71.72,210.50,4.57,36.87,18.43,124.79,6.87,4.14,5-11
4,2014,S,72.00,203.93,4.54,34.83,15.00,118.00,7.14,4.27,6-0
...,...,...,...,...,...,...,...,...,...,...,...
213,2020,OL,76.41,316.20,5.17,29.52,26.38,108.37,7.89,4.81,6-4
214,2021,OL,76.35,313.62,5.14,30.28,28.23,108.87,7.63,4.64,6-4
215,2019,DL,75.54,296.54,4.99,31.28,26.27,110.92,7.54,4.54,6-3
216,2020,DL,75.50,286.06,4.90,31.00,25.00,116.35,7.53,4.55,6-3


### Save each so we can use them later

In [25]:
undrafted_averages.to_csv('data/undrafted_player_averages_per_year.csv', index=False)
drafted_averages.to_csv('data/drafted_player_averages_per_year.csv', index=False)

### Combine the 2 to have averages for undrafted and drafted together in 1 dataframe

In [26]:
undrafted_averages['Drafted'] = 0
drafted_averages['Drafted'] = 1

all_players_df = pd.concat([undrafted_averages, drafted_averages])
all_players_df

,Year,Position,Height_in_inches,Weight,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Height,Drafted
0,2010,S,71.91,208.64,4.59,34.44,15.89,118.30,6.98,4.25,5-11,0
1,2011,S,71.75,207.62,4.56,33.92,19.20,118.17,6.91,4.19,5-11,0
2,2012,S,71.90,206.50,4.56,34.40,17.78,121.40,7.00,4.24,5-11,0
3,2013,S,72.17,210.33,4.62,35.40,16.00,121.80,6.87,4.28,6-0,0
4,2014,S,72.00,208.20,4.66,34.30,16.00,119.00,6.91,4.23,6-0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
213,2020,OL,76.41,316.20,5.17,29.52,26.38,108.37,7.89,4.81,6-4,1
214,2021,OL,76.35,313.62,5.14,30.28,28.23,108.87,7.63,4.64,6-4,1
215,2019,DL,75.54,296.54,4.99,31.28,26.27,110.92,7.54,4.54,6-3,1
216,2020,DL,75.50,286.06,4.90,31.00,25.00,116.35,7.53,4.55,6-3,1


In [28]:
all_players_df.to_csv('data/players_per_position_per_year_averages.csv', index=False)